NOTEBOOK DE SAGEMAKER: SE CONECTA A REDSHIFT, Y CARGA LOS RESULTADOS A OTRA TABLA, TAMBIEN EN REDSHIFT. DE AHI PASAN A QUICKSIGHT COMO PARTE DEL DASHBOARD. 

------------------------------
TOPIC MODELLING PARA review_comments , separados por review_score

Tipo de modelo de ML UNSUPERVISED

VERSION 1, SIN SUBCLASE DE stemmer. (devuelve practicamente mismo rsltds que el 'NewLDA')

Output a pd.Dataframe. 

VER ELECCION PARAMETROS.  version final esta en: ngram_range=(2,4), n_topics=3, n_top_words=3. 

In [ ]:
import redshift_connector
import pandas as pd

conn = redshift_connector.connect(
     host='redshift-cluster-1.cpop3m8tfd5a.us-east-1.redshift.amazonaws.com',
     database='olistdb',
     port=5439,
     user='awsuser',
     password='JldMVsOw&7xL'
  )

conn.autocommit = True

cursor = conn.cursor()

with conn.cursor() as cursor:
    cursor.execute("SELECT review_id, order_id, review_score, review_comment_message, review_creation_date FROM \"olistdb\".\"public\".\"order_reviews\";")
    Review: pd.DataFrame = cursor.fetch_dataframe()

cursor.close()
conn.close()

In [1]:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline
#
import nltk
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
nltk.download('stopwords')
#import Stemmer
#Para traducir la salida: 
#from deep_translator import GoogleTranslator
#import matplotlib as plt
import pandas as pd
import matplotlib.pyplot as plt

#hay varios q no se usan, del otro archivo. limpiar. 


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/moncho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# preparamos los datos que usamos:
#Review = pd.read_csv('Datasets/olist_order_reviews_dataset.csv')

#VER FILTRO POR FECHA: 
#si lo filtra con queries o aca. 

# tiro nulos por 'comment_message': 
Review.dropna(subset = 'review_comment_message', inplace= True)

# me quedo solo con estas cols: TMB AGREGO FECHA A VER SI SE PUEDE PLOTTEAR
Review = Review[['review_score', 'review_comment_message', 'review_id','review_creation_date']] #ie solo tiro review_score

# los separo por score, para poder sacarle los 'topic' por separado a cada uno:
estrella_1 = Review[Review['review_score'] == 1]
estrella_2 = Review[Review['review_score'] == 2]
estrella_3 = Review[Review['review_score'] == 3]
estrella_4 = Review[Review['review_score'] == 4]
estrella_5 = Review[Review['review_score'] == 5]
#de cada uno me quedo con la lista de mensajes: 
comments1=estrella_1['review_comment_message'] 
comments2=estrella_2['review_comment_message']  
comments3=estrella_3['review_comment_message']   
comments4=estrella_4['review_comment_message']   
comments5=estrella_5['review_comment_message'] 
#los meto en una lista para poder iterar: 
comentarios=[comments1, comments2, comments3, comments4, comments5]

In [7]:
# lista STOPWORDS: 

stoplist_ampliada = stopwords.words('portuguese') + ['produto', 'satisfeita', 'n','recomendo', 'recom']
# VER: agregar el 'aun' (en pt)?

#veo de quitarle el 'no', asi arma mas frases con sentido de negacion...
stoplist_ampliada.remove('não')

In [59]:

#iteramos sobre lista 'comentarios' y guardamos todo en un diccionario para pasarlo a DataFrame, y luego a Redshift:
dict={} 
#OBS: PARAMETROS: 
# los de TfidfVectorizer: stop_words=stoplist_ampliada, ngram_range=(2,4),min_df=5, max_df=0.7)
# n_top_words y n_components

for i, comment in enumerate (comentarios, 1): #lo inicio en i=1.

    tfidf_vectorizer = TfidfVectorizer(stop_words=stoplist_ampliada, ngram_range=(2,4),min_df=5, max_df=0.7) 
    lda = LatentDirichletAllocation(n_components=3)   #n_components=3 ?
    pipe = make_pipeline(tfidf_vectorizer, lda)

    # instancio traductor:
    #traductor = GoogleTranslator(source='pt', target='es')

    pipe.fit(comment) 

    lista=[]
    n_top_words=3   #n_top_words=3
    
    for topic_idx, topic in enumerate(lda.components_):
        message = ''    #"Topic #%d: " % topic_idx  #le saco esto para que no moleste en el df. 
        message += ", ".join([tfidf_vectorizer.get_feature_names()[i]  #traductor.translate(tfidf_vectorizer.get_feature_names()[i])
                                    for i in topic.argsort()[:-n_top_words - 1:-1]])
        
        lista.append(message)
    dict.update({'score '+str(i):lista})
           

/Users/moncho/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [62]:
topics_redshift=pd.DataFrame(dict)

In [63]:
#topics_redshift.head()

,score 1,score 2,score 3,score 4,score 5
0,"não recebi, ainda não, não chegou","não gostei, veio defeito, não chegou","não veio, prazo entrega, tudo certo","entregue prazo, tudo ok, ainda não","antes prazo, entregue antes, bem antes"
1,"não entregue, não gostei, dinheiro volta","não recebi, ainda não, ainda não recebi","não recebi, ainda não, dentro prazo","antes prazo, chegou antes, tudo certo","antes prazo, entrega rápida, chegou antes"
2,"agora não, recebi apenas, não veio","não veio, nota fiscal, antes prazo","antes prazo, não gostei, não chegou","dentro prazo, entrega rápida, boa qualidade","ótima qualidade, boa qualidade, entrega super"


In [ ]:
conn = redshift_connector.connect(
     host='redshift-cluster-1.cpop3m8tfd5a.us-east-1.redshift.amazonaws.com',
     database='olistdb',
     port=5439,
     user='awsuser',
     password='JldMVsOw&7xL'
  )

conn.autocommit = True

cursor = conn.cursor()

with conn.cursor() as cursor:
    cursor.execute('create table public.ml_lda_topics (score_1 varchar(500), score_2 varchar(500), score_3 varchar(500), score_4 varchar(500), score_5 varchar(500));')
    cursor.execute('SELECT * FROM public.ml_lda_topics;')
    result = cursor.fetchall()
print(result)

with conn.cursor() as cursor:
    cursor.write_dataframe(topics_redshift, "ml_lda_topics")
    cursor.execute('SELECT * FROM "public"."ml_lda_topics";')
    result: pd.DataFrame = cursor.fetch_dataframe()

cursor.close()
conn.close()